Найти датасет ( можно посмотреть на kaggle, PapersWithCode или sklearn), обучить на нем линейную или логистическую регрессию. Так же необходимо посчитать метрики у обученной модели.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('Cancer_Data.csv')

In [4]:
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


570 раковых клеток и 30 признаков, позволяющих определить, являются ли раковые клетки в данных доброкачественными или злокачественными

Наши данные о раке содержат 2 типа раковых заболеваний: 1. доброкачественный рак (B) и 2. злокачественный рак (M).
    

Удаляем ненужные столбцы (ID, Unnamed:32)

In [5]:
df = df.drop(['id', 'Unnamed: 32'], axis = 1)

Проверяем, есть ли нулевые значения

In [21]:
df.isna().sum()

diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64

Супер, их нет:)

Удаляем колонки, которые сильно отличаются от данных

In [42]:
threshold = 5

for column in df.loc[:, ~df.columns.isin(['diagnosis'])]:  
    
    mean = df[column].mean()
    std = df[column].std()

    lower_limit = mean - threshold * std
    upper_limit = mean + threshold * std

    df = df[(df[column] >= lower_limit) & (df[column] <= upper_limit)]

Переведем колонку diagnosis во множество {0, 1} (0 - B, 1 - M)

In [43]:
df['diagnosis'] = df['diagnosis'].apply(lambda x: 0 if x == 'B' else 1)

In [44]:
df.head(5)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.2,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
5,1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,0.2087,...,15.47,23.75,103.4,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440


Определяем целевую переменную

In [45]:
X = df.drop('diagnosis', axis = 1)
y = df['diagnosis']

Разделение на обучающую и тестовую выборки

In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [47]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, stratify = y, test_size = 0.25, random_state = 42)

In [48]:
# обучение модели
model = LinearRegression()
model.fit(xtrain, ytrain)

LinearRegression()

In [51]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

features = xtest
y_pred = model.predict(features) # предсказание
y_true = ytest # истина

print(f"MSE = {mean_squared_error(y_true, y_pred)}")
print(f"MAE = {mean_absolute_error(y_pred, y_true)}")
print(f"r2_score  = {r2_score(y_true, y_pred)}")

MSE = 0.0
MAE = 0.0
r2_score  = 1.0


In [52]:
df = pd.DataFrame(y_true.copy())
df['prediction'] = y_pred
df.head(15)

,diagnosis,prediction
321,1,1.0
57,1,1.0
449,1,1.0
44,1,1.0
58,1,1.0
47,1,1.0
409,1,1.0
293,1,1.0
205,1,1.0
489,1,1.0


In [53]:
pred = model.predict(xtest)

In [54]:
accuracy = accuracy_score(ytest, pred)
precision = precision_score(ytest, pred)
recall = recall_score(ytest, pred)
f1 = f1_score(ytest, pred)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
